In [2]:
# Importing necessary modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import warnings
warnings.filterwarnings("ignore")

In [3]:
amount = pd.read_csv("Amount_Variables_Final.csv")
frequency= pd.read_csv("Frequency_Variables_Final.csv")
days = pd.read_csv("Dayssince_Variables_Final.csv")
freq_change = pd.read_csv("Velocity_Change_Variables_Final.csv")

In [4]:
amount.shape

(96397, 210)

In [5]:
frequency.shape

(96397, 35)

In [6]:
days.shape

(96397, 15)

In [7]:
freq_change.shape

(96397, 58)

In [8]:
frequency = frequency.iloc[:,10:210]
days = days.iloc[:,10:15]
freq_change = freq_change.iloc[:,10:58]

In [9]:
df1 = pd.concat([amount, frequency,freq_change,days], axis=1, join='inner')
df1.head()

,Date,Recnum,Cardnum,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud,...,rel_vel_amt_merch_merch_30,rel_amt_vel_card_card_30,rel_amt_vel_card_merch_30,rel_amt_amt_card_card_30,rel_amt_amt_card_merch_30,dayssince_card,dayssince_merch,dayssince_cardmerch,dayssince_cardzip,dayssince_cardstate
0,1/1/2010,1,5142190439,5510000000000,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,0,...,0.276243,3.62,3.620000,1.0,1.000000,337,345,345,345,345
1,1/1/2010,2,5142183973,61003026333,SERVICE MERCHANDISE #81,MA,1803,P,31.42,0,...,0.031827,31.42,31.420000,1.0,1.000000,337,345,345,345,345
2,1/1/2010,3,5142131721,4500000000000,OFFICE DEPOT #191,MD,20706,P,178.49,0,...,0.005603,178.49,178.490000,1.0,1.000000,337,345,345,345,345
3,1/1/2010,4,5142148452,5510000000000,FEDEX SHP 12/28/09 AB#,TN,38118,P,3.62,0,...,0.276243,3.62,1.810000,1.0,0.500000,337,365,345,345,345
4,1/1/2010,5,5142190439,5510000000000,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,0,...,0.276243,3.62,2.413333,1.0,0.666667,357,365,365,365,365


In [10]:
df1['Date'] = pd.to_datetime(df1['Date'])
df1['Cardnum'] = df1['Cardnum'].astype(str)
df1['Merch zip'] = df1['Merch zip'].astype(str)

import re
df1 = df1.rename(columns=lambda x: re.sub('1d','1',x))
df1 = df1.rename(columns=lambda x: re.sub('3d','3',x))
df1 = df1.rename(columns=lambda x: re.sub('7d','7',x))
df1 = df1.rename(columns=lambda x: re.sub('14d','14',x))
df1 = df1.rename(columns=lambda x: re.sub('30d','30',x))
df1.head()

,Date,Recnum,Cardnum,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud,...,rel_vel_amt_merch_merch_30,rel_amt_vel_card_card_30,rel_amt_vel_card_merch_30,rel_amt_amt_card_card_30,rel_amt_amt_card_merch_30,dayssince_card,dayssince_merch,dayssince_cardmerch,dayssince_cardzip,dayssince_cardstate
0,2010-01-01,1,5142190439,5510000000000,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,0,...,0.276243,3.62,3.620000,1.0,1.000000,337,345,345,345,345
1,2010-01-01,2,5142183973,61003026333,SERVICE MERCHANDISE #81,MA,1803,P,31.42,0,...,0.031827,31.42,31.420000,1.0,1.000000,337,345,345,345,345
2,2010-01-01,3,5142131721,4500000000000,OFFICE DEPOT #191,MD,20706,P,178.49,0,...,0.005603,178.49,178.490000,1.0,1.000000,337,345,345,345,345
3,2010-01-01,4,5142148452,5510000000000,FEDEX SHP 12/28/09 AB#,TN,38118,P,3.62,0,...,0.276243,3.62,1.810000,1.0,0.500000,337,365,345,345,345
4,2010-01-01,5,5142190439,5510000000000,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,0,...,0.276243,3.62,2.413333,1.0,0.666667,357,365,365,365,365


In [11]:
df1.shape

(96397, 288)

In [12]:
# Creating day of week target encoding (excluding OOT)
df1['dayofweek'] = df1['Date'].dt.day_name()
risk = {'Monday': 0.0087,
        'Tuesday':0.0071,
        'Wednesday':0.0098,
        'Thursday':0.0186,
        'Friday':0.0260,
        'Saturday':0.0101,
        'Sunday':0.0096}
df1['risk_dayofweek'] = df1['dayofweek'].map(risk)
df1.head()

,Date,Recnum,Cardnum,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud,...,rel_amt_vel_card_merch_30,rel_amt_amt_card_card_30,rel_amt_amt_card_merch_30,dayssince_card,dayssince_merch,dayssince_cardmerch,dayssince_cardzip,dayssince_cardstate,dayofweek,risk_dayofweek
0,2010-01-01,1,5142190439,5510000000000,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,0,...,3.620000,1.0,1.000000,337,345,345,345,345,Friday,0.026
1,2010-01-01,2,5142183973,61003026333,SERVICE MERCHANDISE #81,MA,1803,P,31.42,0,...,31.420000,1.0,1.000000,337,345,345,345,345,Friday,0.026
2,2010-01-01,3,5142131721,4500000000000,OFFICE DEPOT #191,MD,20706,P,178.49,0,...,178.490000,1.0,1.000000,337,345,345,345,345,Friday,0.026
3,2010-01-01,4,5142148452,5510000000000,FEDEX SHP 12/28/09 AB#,TN,38118,P,3.62,0,...,1.810000,1.0,0.500000,337,365,345,345,345,Friday,0.026
4,2010-01-01,5,5142190439,5510000000000,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,0,...,2.413333,1.0,0.666667,357,365,365,365,365,Friday,0.026


In [13]:
# Creating Benford Variables
data = df1.copy()
data2 = df1.copy()

data.drop(data[data['Merch description'].str.contains('FEDEX',regex = False)].index,inplace = True)
data.drop(data[data['Merch description'].str.contains('FED EX',regex = False)].index,inplace = True)
data.drop(data[data['Merch description'].str.contains('FEDERAL EXPRESS',regex = False)].index,inplace = True)
data.drop(data[data['Merch description'].str.contains('FEDERAL EX',regex = False)].index,inplace = True)

data2.drop(data[data['Merch description'].str.contains('FEDEX',regex = False)].index,inplace = True)
data2.drop(data[data['Merch description'].str.contains('FED EX',regex = False)].index,inplace = True)
data2.drop(data[data['Merch description'].str.contains('FEDERAL EXPRESS',regex = False)].index,inplace = True)
data2.drop(data[data['Merch description'].str.contains('FEDERAL EX',regex = False)].index,inplace = True)


# Cardnumber
data['Amount']=data['Amount'].astype('str')
grouped_df=data.groupby('Cardnum')['Amount']
gb=grouped_df.groups

c=pd.DataFrame()
e=pd.DataFrame()
for key,values in gb.items():
    d=pd.DataFrame(data.loc[values]['Amount'].transform(lambda x:x.startswith('1')).cumsum())
    c=c.append(d)
    f=pd.DataFrame(data.loc[values]['Amount'].transform(lambda x:x.startswith('2')).cumsum())
    e=e.append(f)

both=c.merge(e,right_index=True,left_index=True)
both.columns=['ones','twos']
both['n_low']=both['ones']+both['twos']
both['n']=data.groupby('Cardnum')['Amount'].cumcount()+1
data=both.merge(data,right_index=True,left_index=True)
data['n_high']=data['n']-data['n_low']
data['n_low']=np.where(data['n_low']==0,1,data['n_low'])
data['n_high']=np.where(data['n_high']==0,1,data['n_high'])
data['R']=data['n_low']*1.096/data['n_high']
data['1/R']=1/(data['R'])
data['U']=data[['R','1/R']].max(axis=1)
data['t']=(data['n']-15)/3
data['U*']=(1+(data['U']-1)/(1+np.exp(-data['t'])))

# Merchant Number
data2['Amount']=data2['Amount'].astype('str')
grouped_df=data2.groupby('Merchnum')['Amount']
gb=grouped_df.groups

c=pd.DataFrame()
e=pd.DataFrame()
for key,values in gb.items():
    d=pd.DataFrame(data2.loc[values]['Amount'].transform(lambda x:x.startswith('1')).cumsum())
    c=c.append(d)
    f=pd.DataFrame(data2.loc[values]['Amount'].transform(lambda x:x.startswith('2')).cumsum())
    e=e.append(f)

both=c.merge(e,right_index=True,left_index=True)
both.columns=['ones','twos']
both['n_low']=both['ones']+both['twos']
both['n']=data2.groupby('Merchnum')['Amount'].cumcount()+1
data2=both.merge(data2,right_index=True,left_index=True)
data2['n_high']=data2['n']-data2['n_low']
data2['n_low']=np.where(data2['n_low']==0,1,data2['n_low'])
data2['n_high']=np.where(data2['n_high']==0,1,data2['n_high'])
data2['R']=data2['n_low']*1.096/data2['n_high']
data2['1/R']=1/(data2['R'])
data2['U']=data2[['R','1/R']].max(axis=1)
data2['t']=(data2['n']-15)/3
data2['U*']=(1+(data2['U']-1)/(1+np.exp(-data2['t'])))

df1 = df1.merge(data[['Recnum','U*']],how='left')
df1 = df1.rename(columns={'U*': 'card_U*'})
df1 = df1.merge(data2[['Recnum','U*']],how='left')
df1 = df1.rename(columns={'U*': 'merch_U*'})

In [14]:
df1.head()

,Date,Recnum,Cardnum,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud,...,rel_amt_amt_card_merch_30,dayssince_card,dayssince_merch,dayssince_cardmerch,dayssince_cardzip,dayssince_cardstate,dayofweek,risk_dayofweek,card_U*,merch_U*
0,2010-01-01,1,5142190439,5510000000000,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,0,...,1.000000,337,345,345,345,345,Friday,0.026,NaN,1.000894
1,2010-01-01,2,5142183973,61003026333,SERVICE MERCHANDISE #81,MA,1803,P,31.42,0,...,1.000000,337,345,345,345,345,Friday,0.026,1.000894,1.000894
2,2010-01-01,3,5142131721,4500000000000,OFFICE DEPOT #191,MD,20706,P,178.49,0,...,1.000000,337,345,345,345,345,Friday,0.026,1.000894,1.000894
3,2010-01-01,4,5142148452,5510000000000,FEDEX SHP 12/28/09 AB#,TN,38118,P,3.62,0,...,0.500000,337,365,345,345,345,Friday,0.026,NaN,1.010684
4,2010-01-01,5,5142190439,5510000000000,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,0,...,0.666667,357,365,365,365,365,Friday,0.026,NaN,1.031246


In [15]:
# Dropping original variables
original_variables = ['Cardnum','Merchnum','Merch description',
                      'Merch state','Merch zip','Transtype','dayofweek','Amount']

df1.drop(original_variables,axis = 1,inplace = True)
df1.head()

,Date,Recnum,Fraud,mean_amt_card_1,max_amt_card_1,median_amt_card_1,sum_amt_card_1,mean_amt_card_3,max_amt_card_3,median_amt_card_3,...,rel_amt_amt_card_card_30,rel_amt_amt_card_merch_30,dayssince_card,dayssince_merch,dayssince_cardmerch,dayssince_cardzip,dayssince_cardstate,risk_dayofweek,card_U*,merch_U*
0,2010-01-01,1,0,3.62,3.62,3.62,3.62,3.62,3.62,3.62,...,1.0,1.000000,337,345,345,345,345,0.026,NaN,1.000894
1,2010-01-01,2,0,31.42,31.42,31.42,31.42,31.42,31.42,31.42,...,1.0,1.000000,337,345,345,345,345,0.026,1.000894,1.000894
2,2010-01-01,3,0,178.49,178.49,178.49,178.49,178.49,178.49,178.49,...,1.0,1.000000,337,345,345,345,345,0.026,1.000894,1.000894
3,2010-01-01,4,0,3.62,3.62,3.62,3.62,3.62,3.62,3.62,...,1.0,0.500000,337,365,345,345,345,0.026,NaN,1.010684
4,2010-01-01,5,0,3.62,3.62,3.62,7.24,3.62,3.62,3.62,...,1.0,0.666667,357,365,365,365,365,0.026,NaN,1.031246


In [16]:
# moving fraud_label to last
df1['fraud_label1'] = df1['Fraud']*1
df1.drop('Fraud',axis = 1,inplace = True)
df1.rename(columns={'fraud_label1': 'Fraud'},inplace = True)
pd.set_option('display.max_columns', None)
df1.head()

,Date,Recnum,mean_amt_card_1,max_amt_card_1,median_amt_card_1,sum_amt_card_1,mean_amt_card_3,max_amt_card_3,median_amt_card_3,sum_amt_card_3,mean_amt_card_7,max_amt_card_7,median_amt_card_7,sum_amt_card_7,mean_amt_card_14,max_amt_card_14,median_amt_card_14,sum_amt_card_14,mean_amt_card_30,max_amt_card_30,median_amt_card_30,sum_amt_card_30,mean_amt_merch_1,max_amt_merch_1,median_amt_merch_1,sum_amt_merch_1,mean_amt_merch_3,max_amt_merch_3,median_amt_merch_3,sum_amt_merch_3,mean_amt_merch_7,max_amt_merch_7,median_amt_merch_7,sum_amt_merch_7,mean_amt_merch_14,max_amt_merch_14,median_amt_merch_14,sum_amt_merch_14,mean_amt_merch_30,max_amt_merch_30,median_amt_merch_30,sum_amt_merch_30,mean_amt_cardmerch_1,max_amt_cardmerch_1,median_amt_cardmerch_1,sum_amt_cardmerch_1,mean_amt_cardmerch_3,max_amt_cardmerch_3,median_amt_cardmerch_3,sum_amt_cardmerch_3,mean_amt_cardmerch_7,max_amt_cardmerch_7,median_amt_cardmerch_7,sum_amt_cardmerch_7,mean_amt_cardmerch_14,max_amt_cardmerch_14,median_amt_cardmerch_14,sum_amt_cardmerch_14,mean_amt_cardmerch_30,max_amt_cardmerch_30,median_amt_cardmerch_30,sum_amt_cardmerch_30,mean_amt_cardzip_1,max_amt_cardzip_1,median_amt_cardzip_1,sum_amt_cardzip_1,mean_amt_cardzip_3,max_amt_cardzip_3,median_amt_cardzip_3,sum_amt_cardzip_3,mean_amt_cardzip_7,max_amt_cardzip_7,median_amt_cardzip_7,sum_amt_cardzip_7,mean_amt_cardzip_14,max_amt_cardzip_14,median_amt_cardzip_14,sum_amt_cardzip_14,mean_amt_cardzip_30,max_amt_cardzip_30,median_amt_cardzip_30,sum_amt_cardzip_30,mean_amt_cardstate_1,max_amt_cardstate_1,median_amt_cardstate_1,sum_amt_cardstate_1,mean_amt_cardstate_3,max_amt_cardstate_3,median_amt_cardstate_3,sum_amt_cardstate_3,mean_amt_cardstate_7,max_amt_cardstate_7,median_amt_cardstate_7,sum_amt_cardstate_7,mean_amt_cardstate_14,max_amt_cardstate_14,median_amt_cardstate_14,sum_amt_cardstate_14,mean_amt_cardstate_30,max_amt_cardstate_30,median_amt_cardstate_30,sum_amt_cardstate_30,ratio_actual_mean_amt_card_1,ratio_actual_mean_amt_merch_1,ratio_actual_mean_amt_cardmerch_1,ratio_actual_mean_amt_cardzip_1,ratio_actual_mean_amt_cardstate_1,ratio_actual_max_amt_card_1,ratio_actual_max_amt_merch_1,ratio_actual_max_amt_cardmerch_1,ratio_actual_max_amt_cardzip_1,ratio_actual_max_amt_cardstate_1,ratio_actual_median_amt_card_1,ratio_actual_median_amt_merch_1,ratio_actual_median_amt_cardmerch_1,ratio_actual_median_amt_cardzip_1,ratio_actual_median_amt_cardstate_1,ratio_actual_sum_amt_card_1,ratio_actual_sum_amt_merch_1,ratio_actual_sum_amt_cardmerch_1,ratio_actual_sum_amt_cardzip_1,ratio_actual_sum_amt_cardstate_1,ratio_actual_mean_amt_card_3,ratio_actual_mean_amt_merch_3,ratio_actual_mean_amt_cardmerch_3,ratio_actual_mean_amt_cardzip_3,ratio_actual_mean_amt_cardstate_3,ratio_actual_max_amt_card_3,ratio_actual_max_amt_merch_3,ratio_actual_max_amt_cardmerch_3,ratio_actual_max_amt_cardzip_3,ratio_actual_max_amt_cardstate_3,ratio_actual_median_amt_card_3,ratio_actual_median_amt_merch_3,ratio_actual_median_amt_cardmerch_3,ratio_actual_median_amt_cardzip_3,ratio_actual_median_amt_cardstate_3,ratio_actual_sum_amt_card_3,ratio_actual_sum_amt_merch_3,ratio_actual_sum_amt_cardmerch_3,ratio_actual_sum_amt_cardzip_3,ratio_actual_sum_amt_cardstate_3,ratio_actual_mean_amt_card_7,ratio_actual_mean_amt_merch_7,ratio_actual_mean_amt_cardmerch_7,ratio_actual_mean_amt_cardzip_7,ratio_actual_mean_amt_cardstate_7,ratio_actual_max_amt_card_7,ratio_actual_max_amt_merch_7,ratio_actual_max_amt_cardmerch_7,ratio_actual_max_amt_cardzip_7,ratio_actual_max_amt_cardstate_7,ratio_actual_median_amt_card_7,ratio_actual_median_amt_merch_7,ratio_actual_median_amt_cardmerch_7,ratio_actual_median_amt_cardzip_7,ratio_actual_median_amt_cardstate_7,ratio_actual_sum_amt_card_7,ratio_actual_sum_amt_merch_7,ratio_actual_sum_amt_cardmerch_7,ratio_actual_sum_amt_cardzip_7,ratio_actual_sum_amt_cardstate_7,ratio_actual_mean_amt_card_14,ratio_actual_mean_amt_merch_14,ratio_actual_mean_amt_cardmerch_14,ratio_actual_mean_amt_cardzip_14,ratio_actual_mean_amt_cardstat

In [17]:
df1.fillna(1, inplace=True)
df1.isnull().sum()

Date                   0
Recnum                 0
mean_amt_card_1        0
max_amt_card_1         0
median_amt_card_1      0
                      ..
dayssince_cardstate    0
risk_dayofweek         0
card_U*                0
merch_U*               0
Fraud                  0
Length: 284, dtype: int64

In [18]:
a = list(df1.columns)
a

['Date',
 'Recnum',
 'mean_amt_card_1',
 'max_amt_card_1',
 'median_amt_card_1',
 'sum_amt_card_1',
 'mean_amt_card_3',
 'max_amt_card_3',
 'median_amt_card_3',
 'sum_amt_card_3',
 'mean_amt_card_7',
 'max_amt_card_7',
 'median_amt_card_7',
 'sum_amt_card_7',
 'mean_amt_card_14',
 'max_amt_card_14',
 'median_amt_card_14',
 'sum_amt_card_14',
 'mean_amt_card_30',
 'max_amt_card_30',
 'median_amt_card_30',
 'sum_amt_card_30',
 'mean_amt_merch_1',
 'max_amt_merch_1',
 'median_amt_merch_1',
 'sum_amt_merch_1',
 'mean_amt_merch_3',
 'max_amt_merch_3',
 'median_amt_merch_3',
 'sum_amt_merch_3',
 'mean_amt_merch_7',
 'max_amt_merch_7',
 'median_amt_merch_7',
 'sum_amt_merch_7',
 'mean_amt_merch_14',
 'max_amt_merch_14',
 'median_amt_merch_14',
 'sum_amt_merch_14',
 'mean_amt_merch_30',
 'max_amt_merch_30',
 'median_amt_merch_30',
 'sum_amt_merch_30',
 'mean_amt_cardmerch_1',
 'max_amt_cardmerch_1',
 'median_amt_cardmerch_1',
 'sum_amt_cardmerch_1',
 'mean_amt_cardmerch_3',
 'max_amt_cardmerch

In [17]:
# df1.to_csv('Final Variable Data.csv')